# eBay Web Scraping
# Jing Hao

In [1]:
import urllib.request, urllib.parse, urllib.error
import requests
import json
import sqlite3
import re
import os
import os.path
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

## Part 1

### a) Write a program that accesses all trivia results for the numbers from 0 (zero) to 99 using batch requests only (One single query for all the numbers). Print the output result to the screen in the format of [3-digit number with leading zeros] - [TRIVIA] 

In [2]:
serviceurl = "http://numbersapi.com/"
number = "0..99"

In [3]:
# create the url to get data
url = serviceurl + number
# get and open url
uh = urllib.request.urlopen(url)
# read data
data = uh.read()
# load data using json
json_data=json.loads(data)

In [4]:
# define a function to print json data in required format
def print_json(json_data):
    for key in list(json_data.keys()):
        print(f"{str(key).zfill(3)} - {json_data[key]}")

## Part 2: Web-scraping: Which sellers advertise/sponsor on eBay

In [7]:
def saveTxt(filename, file_content):
    with open(filename + ".txt", "w") as f:
        for row in file_content:
            f.write(row + '\n')

In [8]:
ebay_url = "https://www.ebay.com/sch/i.html?_from=R40&_nkw=playstation+4+slim&_sacat=0&rt=nc&LH_BIN=1&_ipg=100"
html = urllib.request.urlopen(ebay_url).read()

In [9]:
# set the url pattern that determine the page number
page_pat = "&_pgn="
# Set the Sponsored pattern
pattern = ".*S.*P.*O.*N.*S.*O.*R.*E.*D"
# create the list to store the sponsored and non-sponsored links for ten pages
tenPage_sponsored = []
tenPage_non_sponsored = []

for i in range(1, 11):
    # get the url for the ith page
    new_url = ebay_url + page_pat + str(i)
    # get the content of ith page
    html_content = urllib.request.urlopen(new_url).read()
    # store the page content in 'soup'
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # create lists to store the urls of the sponsored and non-sponsored items in this page
    singlePage_sponsored = []
    singlePage_non_sponsored = []
    # class .s-item__link store item information
    # get the infomation of all items
    items = soup.select("#srp-river-results > ul > li.s-item")
    
    for item in items:
        a = item.find("a")
        if not a:
            continue
        txt = item.find("span").get_text()
        result = bool(re.search(pattern, txt))
        link = a['href']
    
        # to determine if it is sponsored
        if (result == True):
            link_sponsored = link
            # append the link of sponsored item in this page
            singlePage_sponsored.append(link_sponsored)
        else:
            link_non_sponsored = link
            # append the link of non-sponsored item in this page
            singlePage_non_sponsored.append(link_non_sponsored)
    
    # append the links of sponsored and non-sponsored items in each page to a list
    tenPage_sponsored += singlePage_sponsored   
    tenPage_non_sponsored += singlePage_non_sponsored

In [12]:
# save urls to corresponding folders
sponsored = 'sponsored'
non_sponsored = 'non-sponsored'
saveTxt(sponsored, tenPage_sponsored)
saveTxt(non_sponsored, tenPage_non_sponsored)

In [13]:
# define a function to create the folder
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [14]:
# create the folder
createFolder('./sponsored/')
createFolder('./non_sponsored/')

In [15]:
# save urls in a txt file respectively

sponsored_url = []
non_sponsored_url = []

with open('sponsored.txt') as sponsored:
    for line in sponsored:
        sponsored_url.append(line.rstrip('\n'))
        
with open('non-sponsored.txt') as non_sponsored:
    for line in non_sponsored:
        non_sponsored_url.append(line.rstrip('\n'))

In [16]:
for url in sponsored_url:
    html_content = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html_content,'html.parser')
    
    # find all sponsored products' id 
    prodDetail = soup.find('div', {'class': 'prodDetailSec'})
    if not prodDetail:
        continue
    prodId_txt = prodDetail.find('td', text = 'eBay Product ID (ePID)')
    prodId = prodId_txt.findNext('td').text.strip()
    
    # save the html content and name the file using product id
    save_path = './sponsored/'
    completeName = os.path.join(save_path, prodId + ".html")
    with open(completeName , "wb") as f:
        f.write(html_content)
        f.close()
    time.sleep(1)

In [17]:
for url in non_sponsored_url:
    html_content = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html_content,'html.parser')
    
    # find all non-sponsorod products' id
    prodDetail = soup.find('div', {'class': 'prodDetailSec'})
    if not prodDetail:
        continue
    prodId_txt = prodDetail.find('td', text = 'eBay Product ID (ePID)')
    prodId = prodId_txt.findNext('td').text.strip()
    
    # save the html content and name the file using product id
    save_path = './non_sponsored/'
    completeName = os.path.join(save_path, prodId + ".html")
    with open(completeName , "wb") as f:
        f.write(html_content)
        f.close()
    time.sleep(1)

In [18]:
# web scraping to find all the info
seller_name_list = []
seller_score_list = []
item_price_list = []
qty_sold_list = []
best_offer_available_list = []
title_list = []
returns_allowed_list = []
shipping_computed_list = []
condition_list = []
sponsor_list = []

folders = ['./sponsored/', './non_sponsored/']
for folder in folders:
    folder_list = os.listdir(folder)

    for file in folder_list:
        if not file.endswith('html'):
            continue
        fname = os.path.join(folder,file)
        html_file_content = open(fname, 'r', encoding = 'utf-8').read()
        soup_all = BeautifulSoup(html_file_content, 'html.parser')
        
        ### find seller name
        seller = soup_all.find('div',{'class':'mbg vi-VR-margBtm3'}).a.span.text
        seller_name_list.append(seller)
        
        ### find seller score
        if not soup_all.find('a', {'class': 'reviews-star-rating'}):
            score = None
        else:
            score = soup_all.find('a', {'class': 'reviews-star-rating'}).attrs['aria-label'].split()[0]
        seller_score_list.append(score)
        
        ### find item price
        if not soup_all.find('span', {'id': 'prcIsum'}):
            item_price = soup_all.find('span', {'id': 'mm-saleDscPrc'}).attrs['content']
        else: 
            item_price = soup_all.find('span', {'id': 'prcIsum'}).attrs['content']
        item_price_list.append(item_price)
        
        ### find # of items sold
        if not soup_all.find('a', {'class': 'vi-txt-underline'}):
            qty = None
        else:
            qty = soup_all.find('a', {'class': 'vi-txt-underline'}).text.replace(',', '').split()[0]
        qty_sold_list.append(qty)

        ### find best offer available or not
        if not soup_all.find('div', {'class': 'vi-bbox-dspn u-flL lable boLable'}):
            best_offer = 'NO'
        else: best_offer = 'YES'
        best_offer_available_list.append(best_offer)
        
        ### find product title
        title = soup_all.find('h1', {'class': 'it-ttl'}).text
        title_split = re.split(r'\s{2,}', title)[1]
        title_list.append(title_split)

        ### find if return is allowed
        returns_txt = soup_all.find('span', {'id': 'vi-ret-accrd-txt'}).text
        if returns_txt == 'Seller does not accept returns':
            returns = 'NO'
        else:
            returns = 'YES'
        returns_allowed_list.append(returns)

        ### find if shipping cost is computed using zip or not
        if not soup_all.find('input', {'id': 'shGetRates'}):
            shipping_computed = 'NO'
        else:
            shipping_computed = 'YES'
        shipping_computed_list.append(shipping_computed)

        ### find product condition
        condition = soup_all.find('div', {'id': 'vi-itm-cond'}).text
        condition_list.append(condition)

        ### lable if the product is sponsored or not
        if folder == './sponsored/':   
            sponsor = ['YES']
        else:
            sponsor = ['NO'] 
        sponsor_list += sponsor

In [19]:
# convert data to correct format

seller_scores = []
for score in seller_score_list:
    if score != None:
        score_float = float(score)
    else:
        score_float = None
    seller_scores.append(score_float)

item_price_float = []
item_prices = []
for price in item_price_list:
    item_price_float.append(float(price))
for price_float in item_price_float:
    dollar_cent = int(price_float*100)
    item_prices.append(dollar_cent)
    
qty_sold = []
for qty in qty_sold_list:
    if qty != None:
        qty_int = int(qty)
    else: 
        qty_int = None
    qty_sold.append(qty_int)

### Connect to SQL. Create a database and name it "eBay". Save the information into a single table named "eBay_items" 

In [20]:
# connect to SQLite3 and create a database named "eBay.db"
connection = sqlite3.connect('eBay.db')
cursor = connection.cursor()
# create sql query to create table "eBay_items"
create_table = """CREATE TABLE IF NOT EXISTS eBay_items (seller_name TEXT, seller_score REAL, item_price INTEGER,
                qty_sold INTEGER, best_offer_available BLOB, title TEXT, returns_allowed BLOB, shipping_computed BLOB,
                condition TEXT, sponsor TEXT)"""
# execute sql query
cursor.execute(create_table)
# Commits the change and close the connection to the database
connection.commit()
connection.close()

In [ ]:
# connect to the database
conn = sqlite3.connect('eBay.db')
cur = conn.cursor()

for i in range(len(seller_name_list)):

    ### sql commend to insert values to the table
    query = "INSERT INTO eBay_items VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    cur.execute(query, (seller_name_list[i], seller_scores[i], item_prices[i], qty_sold[i], best_offer_available_list[i], title_list[i], 
                        returns_allowed_list[i], shipping_computed_list[i], condition_list[i], sponsor_list[i]))
### print out sql table content
cur.execute("SELECT * FROM eBay_items")
rows = cur.fetchall()
for row in rows:
    print(row)      

# commit changes and close database
conn.commit()
conn.close()

### f) Use code script to run summary stats on each item.

In [22]:
# connect to the database
conn = sqlite3.connect('eBay.db')
cur=conn.cursor()

# convert boolean data to 0 or 1
query_update_offer = "UPDATE eBay_items SET best_offer_available = CASE WHEN best_offer_available = 'YES' THEN 1 ELSE 0 END"
cur.execute(query_update_offer)

query_update_return = "UPDATE eBay_items SET returns_allowed = CASE WHEN returns_allowed = 'YES' THEN 1 ELSE 0 END"
cur.execute(query_update_return)

query_update_shipping = "UPDATE eBay_items SET shipping_computed = CASE WHEN shipping_computed = 'YES' THEN 1 ELSE 0 END"
cur.execute(query_update_shipping)

# sql queries to calculate stats for each column, and print the results out
columns = ['seller_score', 'item_price', 'qty_sold', 'best_offer_available', 'returns_allowed', 'shipping_computed']
for col in columns:
    
    print(col + ' MIN')
    query_min = "SELECT sponsor, condition, MIN(" + str(col) + ") FROM eBay_items WHERE " + str(col) + " IS NOT NULL GROUP BY sponsor, condition;"
    cur.execute(query_min)
    result_min = cur.fetchall()
    for row in result_min:
        print(row)
    print("--------------------------------------")
    
    print(col + " MAX")
    query_max = "SELECT sponsor, condition, MAX(" + str(col)+ ") FROM eBay_items WHERE " + str(col)+ " IS NOT NULL GROUP BY sponsor, condition;"
    cur.execute(query_max)
    result_max = cur.fetchall()
    for row in result_max:
        print(row)
    print("--------------------------------------")
    
    print(col + " MEAN")
    query_mean = "SELECT sponsor, condition, AVG(" + str(col)+ ") FROM eBay_items WHERE " + str(col)+ " IS NOT NULL GROUP BY sponsor, condition;"
    cur.execute(query_mean)
    result_mean = cur.fetchall()
    for row in result_mean:
        print(row)
    print("--------------------------------------")
    
query_count = "SELECT sponsor, condition, COUNT(seller_name), COUNT(title) from eBay_items GROUP BY sponsor, condition;"
cur.execute(query_count)
result_count = cur.fetchall()
for row in result_count:
    print(row)

# commit changes and close database
conn.commit()
conn.close()

seller_score MIN
('NO', 'New', 4.6)
('NO', 'Open box', 5.0)
('NO', 'Seller refurbished', 4.8)
('NO', 'Used', 4.5)
('YES', 'New', 4.7)
('YES', 'Seller refurbished', 4.8)
('YES', 'Used', 4.5)
--------------------------------------
seller_score MAX
('NO', 'New', 5.0)
('NO', 'Open box', 5.0)
('NO', 'Seller refurbished', 4.8)
('NO', 'Used', 4.9)
('YES', 'New', 5.0)
('YES', 'Seller refurbished', 4.8)
('YES', 'Used', 4.8)
--------------------------------------
seller_score MEAN
('NO', 'New', 4.845454545454545)
('NO', 'Open box', 5.0)
('NO', 'Seller refurbished', 4.8)
('NO', 'Used', 4.736363636363635)
('YES', 'New', 4.844444444444444)
('YES', 'Seller refurbished', 4.8)
('YES', 'Used', 4.72)
--------------------------------------
item_price MIN
('NO', 'New', 26211)
('NO', 'Open box', 30000)
('NO', 'Seller refurbished', 20000)
('NO', 'Used', 14000)
('YES', 'New', 26200)
('YES', 'Seller refurbished', 25300)
('YES', 'Used', 7998)
--------------------------------------
item_price MAX
('NO', 'New', 